# Summary Statistics

This notebook displays summary statistics of patient records,
comparing multiple snapshots extracted by the `cohortextractor` action.

## Preliminaries

### Setup

In [ ]:
from IPython.display import display, Markdown
import itertools
import matplotlib
import pandas
import utils

In [ ]:
%matplotlib inline
matplotlib.style.use("seaborn")

### Read data

In [ ]:
records = pandas.read_feather("../output/output.feather")

## Patients

How many patients are in each file?

---
**Aside**: There are several ways to count groups in Pandas. For consistency, we will:

* locate the columns of interest
* group by these columns
* count the number of rows

For readability, we will also put each step on a separate line.

---

In [ ]:
utils.redact_or_display(records
    .loc[:, ["f_name", "patient_id"]]
    .groupby("f_name")
    .count())

How many patients are in each file, by sex?

In [ ]:
utils.redact_or_display(records
    .loc[:, ["f_name", "sex", "patient_id"]]
    .groupby(["f_name", "sex"])
    .count())

How many patients are in each file, by age group?

In [ ]:
by_age_group = (records
    .loc[:, ["age_group", "f_name", "patient_id"]]
    .groupby(["age_group", "f_name"])
    .count())

In [ ]:
utils.redact_or_display(by_age_group.unstack())

In [ ]:
if utils.must_redact(by_age_group):
    print(utils.REDACTION_MSG)
else:
    _ = (by_age_group
        .unstack()
        .plot.bar(subplots=True, figsize=(6, 6), legend=False))

For each combination of files, how many patients are in:

* both files
* the first file but not the second file
* the second file but not the first file

---
**Aside**: We compare each combination of files to future-proof our notebook.
Because order isn't significant, we use `itertools.combinations` rather than `itertools.permutations`.

---

In [ ]:
# Let's revisit sets.
# https://docs.python.org/3.8/library/stdtypes.html#set
set_1 = {1, 2, 3}
set_2 = {3, 4, 5}
assert set_1 & set_2 == {3}  # In set 1 and in set 2 (intersection)
assert set_1 - set_2 == {1, 2}  # In set 1, but not in set 2 (difference)
assert set_2 - set_1 == {4, 5}  # In set 2, but not in set 1 (difference)
del set_1
del set_2

In [ ]:
for f_name_0, f_name_1 in itertools.combinations(records.f_name.cat.categories, 2):
    patient_id_0 = set(records.patient_id[records.f_name == f_name_0])
    patient_id_1 = set(records.patient_id[records.f_name == f_name_1])

    in_0_and_in_1 = len(patient_id_0 & patient_id_1)
    in_0_not_in_1 = len(patient_id_0 - patient_id_1)
    in_1_not_in_0 = len(patient_id_1 - patient_id_0)
    
    if min(in_0_and_in_1, in_0_not_in_1, in_1_not_in_0) <= utils.SMALL_NUMBER:
        print(utils.REDACTION_MSG)
    else:
        display(Markdown(f"{in_0_and_in_1} patients are in *{f_name_0}* and in *{f_name_1}*."))
        display(Markdown(f"{in_0_not_in_1} patients are in *{f_name_0}* but not in *{f_name_1}*."))
        display(Markdown(f"{in_1_not_in_0} patients are in *{f_name_1}* but not in *{f_name_0}*."))

## Practices

How many practices are in each file?

In [ ]:
utils.redact_or_display(records
    .loc[:, ["f_name", "practice_pseudo_id"]]
    .drop_duplicates()  # Remember that rows are patient records.
    .groupby("f_name")
    .count())